In [ ]:
import pyttsx3
import pandas as pd
import datetime

data = pd.read_csv('Reposit.csv').to_dict('records')
n = pyttsx3.init()
voices = n.getProperty('voices')
n.setProperty("rate", 150,)  
n.setProperty("voice",voices[3].id)

current_date = datetime.datetime.now().strftime('%B %m, %Y')
current_time = datetime.datetime.now().strftime("%I %M %p")

print("Great Day! I am a simple Chatbot.")
n.say("こんにちは、私はAIです。どのようにお手伝いできますか？")
n.runAndWait()

while True:
    user_input = input("User: ").lower()  
    found_response = False 

    for d in data:
        if d['phrase'] in user_input:
            print("Uno:", d['response'])
            n.say(d['response'])
            n.runAndWait()
            found_response = True
            break
    if "time" in user_input:
        print("Uno: Current Time", current_time)
        n.say("Current Time"+ current_time)
        n.runAndWait()
        found_response = True
    if "date" in user_input:
        print("Uno: Current Date is ", current_date)
        n.say("Current Date is"+ current_date)
        n.runAndWait()
        found_response = True
        
    if user_input in {"exit", "end", "close", "quit","bye"}:
        print("Uno: Goodbye! Have a nice day!.")
        n.say("Goodbye! Have a nice day!.")
        n.runAndWait()
        break
    
    if not found_response:
        print("Uno: Sorry UwU")
        n.say("Uno: Sorry UwU")
        n.runAndWait()
    
    print("Uno: Anything else?")
    n.say("Anything else?")
    n.runAndWait()
    
n.stop()

In [1]:
import pandas as pd
import datetime
import pyttsx3
import tkinter as tk
import requests
import pyautogui
import time
import speech_recognition as sr
from nltk.corpus import wordnet
from tkinter.simpledialog import askstring


api_key = 'a23ac571904ec5a0e8937a98c291c38c'
data = pd.read_csv('Reposit.csv').to_dict('records')
n = pyttsx3.init()
voices = n.getProperty('voices')
n.setProperty("rate", 150)
n.setProperty("voice", voices[2].id)

current_date = datetime.datetime.now().strftime('%B %d, %Y')
current_time = datetime.datetime.now().strftime("%I:%M %p")

def recognize_speech():
    print("Listening...")
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)

    try:
        print("Recognizing...")
        user_input = recognizer.recognize_google(audio).lower()
        print("You:", user_input)
        input_entry.delete(0, "end")
        input_entry.insert(0, user_input)
        send_message()
    except sr.UnknownValueError:
        print("Could not understand audio.")
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service; {e}")

def calculate(expression):
    allowed_chars = "0123456789+-*/().^"

    if not all(char in allowed_chars for char in expression):
        return "Error: Invalid characters in the expression."

    expression = expression.replace("^", "**")

    try:
        result = eval(expression)
        return result
    except Exception as e:
        return "Error: " + str(e)

def add_chat_message(sender, message, tag):
    chat_display.config(state="normal")
    chat_display.insert("end", f"{sender}: {message}\n", tag)
    chat_display.config(state="disabled")
    chat_display.see("end")

def calculate_days_since(birthday_str):
    try:
        birthday = datetime.datetime.strptime(birthday_str, '%Y-%m-%d')
        today = datetime.datetime.now()
        delta = today - birthday
        return delta.days
    except ValueError:
        return "Invalid date format. Please enter your birthday in YYYY-MM-DD format."
        
def send_message():
    
    user_input = input_entry.get().strip().lower()
    
    if user_input:
        add_chat_message("You", user_input, "user_message")
        input_entry.delete(0, "end")  
        response_text = ""
        found_response = False

        for d in data:
            if d['phrase'] in user_input:
                response_text = d['response']
                found_response = True
                break
        if "time" in user_input:
            response_text = "Current Time: " + current_time
            found_response = True
        if "date" in user_input:
            response_text = "Current Date is " + current_date
            found_response = True

        if "solve" in user_input:
            words = user_input.split()
            if len(words) >= 2 and words[0].lower() == 'solve':
                word_to_define = words[1]
                response = calculate(word_to_define)
                response_text = f"The Answer is {response}"
                found_response = True

        if "birthday" in user_input:
            user_birthday = askstring("Birthday", "Enter your birthday (YYYY-MM-DD):")
            days_since = calculate_days_since(user_birthday)
            response_text = f"Days since your birthday: {days_since} days"
            found_response = True

        if user_input in {"weather"}:
            city = askstring("Weather", "Enter the city:")
            country_code = askstring("Weather", "Enter the country code (e.g., US):")

            if city and country_code:
                url = f'http://api.openweathermap.org/data/2.5/weather?q={city},{country_code}&appid={api_key}'
                response = requests.get(url)
                weather_data = response.json()

                if response.status_code == 200:
                    weather = weather_data['weather'][0]['description']
                    temperature = weather_data['main']['temp'] - 273.15
                    humidity = weather_data['main']['humidity']
                    response_text = f"Weather in {city}, : {weather.capitalize()}\nTemperature: {temperature:.2f}°C\nHumidity: {humidity}%"
                    found_response = True
                else:
                    response_text = "Unable to fetch weather data. Please check your API key and location."
                    
        if user_input in {"exit", "end", "close", "quit", "bye"}:
            response_text = "Goodbye! Have a nice day!"
            found_response = True
            root.destroy()

        if user_input.startswith("count"):
            try:
                count_number = int(user_input.split(" ")[1])
                response_text = "Counting up to {}: {}".format(count_number, ", ".join(map(str, range(1, count_number + 1))))
                found_response = True
            except (ValueError, IndexError):
                response_text = "Invalid count format. Please use 'count <number>'."
        
        if "house" in user_input:
            response_text = "Here you go!"
            found_response = True
            pyautogui.hotkey('winleft')
            pyautogui.write('paint')
            pyautogui.press('enter')
            time.sleep(3)

            pyautogui.moveTo(500, 500)
            pyautogui.click()

            pyautogui.click(830, 80) 
            pyautogui.click(830, 80) 

            pyautogui.moveTo(500, 500)
            pyautogui.dragRel(150, 0) 
            pyautogui.dragRel(0, -100) 
            pyautogui.dragRel(-150, 0)  
            pyautogui.dragRel(0, 100)  

            pyautogui.click(960, 60)  
            pyautogui.click(960, 60)  

            pyautogui.moveTo(500, 400)
            pyautogui.dragRel(75, -75)  
            pyautogui.moveTo(650, 400)
            pyautogui.dragRel(-75, -75) 

            pyautogui.moveTo(270, 70)
            pyautogui.click(270, 70)

            pyautogui.moveTo(550, 385)
            pyautogui.click(550, 385)

            pyautogui.moveTo(830, 60)
            pyautogui.click(830, 60)

            pyautogui.moveTo(560, 450)
            pyautogui.click(560, 450)

            pyautogui.moveTo(895,60)
            pyautogui.click(895,60)     
        
        if "define" in user_input:
            words = user_input.split()
            if len(words) >= 2 and words[0].lower() == 'define':
                word_to_define = words[1]
                synsets = wordnet.synsets(word_to_define)

                if synsets:
                    response_text = synsets[0].definition()
                else:
                    response_text = 'Definition not found for "{word_to_define}" in WordNet.'
            else:
                response_text = 'Invalid command. Please use the format "define word".'
        elif not found_response:
            response_text = "Sorry, I don't understand."
            
        add_chat_message("Uno", response_text, "bot_message")
        chat_display.update_idletasks()
        n.say(response_text)
        n.runAndWait()
        
root = tk.Tk()
root.title("Uno Chatbot")
root.geometry("610x450") 


chat_display = tk.Text(root, wrap="word", state="disabled", width=48, height=20, bg="black",font="Verdana")
chat_display.grid(row=0, column=0, padx=10, pady=10)
chat_display.tag_configure("user_message", foreground="lightblue")
chat_display.tag_configure("bot_message", foreground="lightgreen")

input_entry = tk.Entry(root, width=40, bg="lightblue")
input_entry.grid(row=1, column=0, padx=25, pady=15)
input_entry.bind("<Return>", lambda event=None: send_message())

def on_speak_button_click():
    print("Speak button clicked.")
    recognize_speech()

speech_button = tk.Button(root, text="Speak", command=on_speak_button_click)
speech_button.grid(row=1, column=1, padx=15, pady=15)

root.mainloop()